# La Base permanente des équipements (BPE) 

## La BPE, qu'est-ce que c'est?

La source Base permanente des équipements (BPE) est une base à vocation statistique. Elle répertorie un large éventail d'équipements et de services, marchands ou non, accessibles au public sur l'ensemble de la France au 1ᵉʳ janvier de chaque année.

Elle est construite à partir de sources administratives diverses. Elle rassemble des données sur des points d'accès aux services destinés à la population, ou équipements, localisés à des niveaux géographiques fins : communes, territoires infra-communaux (Iris) et coordonnées `(x,y)`.


Pour en savoir plus - Définitions, méthodes et qualité, consultez :  [https://www.insee.fr/fr/metadonnees/source/serie/s1161](https://www.insee.fr/fr/metadonnees/source/serie/s1161)

## BPE - Données 2023 

Les données utilisées dans cette section sont issues de la BPE de l'INSEE, édition 2023.
Cela correspond aux données disponibles au 1ᵉʳ janvier 2024 (mise à jour en mai 2024), relatives aux équipements et services recensés au 1ᵉʳ janvier 2023. 

## Objectif

Dans cette section, nous allons explorer la BPE, ses variables clés, proposer une méthode de categorisation des équipements par raison sociale en utilisant des outils python (`geopandas`).

In [1]:
# Import biblio necessaire
import geopandas as gpd

## Chargement des données de la BPE

Dans un premier temps, je commence par charger les données et voir ce quelles contiennent en affichant quelques lignes.
La base chargé est celle de 2023 et est dejà filtré sur marseille (actuelle zone d'étude) par un outil QGIS. Cette filtrage sera automatisé prochainement afin de permettre de passer à la BPE general (France) vers une BPE reduit sur Marseille.

In [2]:
# Fichier deses données de la BPE filtré sur Marseille 
data_path = "../../proxy/data/processed/amenities/bpe2023_Marseille.gpkg"
# Chargement des données 
data = gpd.read_file(data_path)

# Affichage des 5 premières lignes
data.head()

,AN,NOMRS,CNOMRS,NUMVOIE,INDREP,TYPVOIE,LIBVOIE,CADR,LIBCOM,CODPOS,...,QUALI_QVA,ZUS,QUALI_ZUS,EPCI,UU2020,BV2022,AAV2020,DENS3,DENS7,geometry
0,2023,DIR REG FIP PACA ET BOUCHES DU RHONE,TRESORERIE ALLAUCH,,,TRA,LA CHAP DES FILLES,,ALLAUCH,13190,...,_Z,CSZ,_Z,200054807,00759,13055,003,1,1,POINT (901350.000 6251550.000)
1,2023,FRANCE SERVICES D’ALLAUCH - PLAN DE CUQUES,,17,,BD,CLEMENT ET JULES BARTHELEMY,,ALLAUCH,13190,...,_Z,CSZ,_Z,200054807,00759,13055,003,1,1,POINT (899921.860 6251277.510)
2,2023,ALLAUCH,,1,,PL,DOCTEUR JOSEPH CHEVILLON,,ALLAUCH,13190,...,_Z,CSZ,_Z,200054807,00759,13055,003,1,1,POINT (901355.700 6251704.700)
3,2023,CAISSE D'EPARGNE CEPAC,AGENCE ALLAUCH CANTON VERT,,,AV,DU CANTON VERT,CC ESPACE MELVILLE,ALLAUCH,13190,...,_Z,CSZ,_Z,200054807,00759,13055,003,1,1,POINT (900461.000 6252323.800)
4,2023,BANQUE POPULAIRE MEDITERRANEE,,,,AV,LEI RIMA,LE LOGIS NEUF,ALLAUCH,13190,...,_Z,CSZ,_Z,200054807,00759,13055,003,1,1,POINT (901698.290 6254031.270)


In [3]:
# Affiche les variables
data.columns

Index(['AN', 'NOMRS', 'CNOMRS', 'NUMVOIE', 'INDREP', 'TYPVOIE', 'LIBVOIE',
       'CADR', 'LIBCOM', 'CODPOS', 'DEPCOM', 'DEP', 'REG', 'DOM', 'SDOM',
       'TYPEQU', 'SIRET', 'STATUT_DIFFUSION', 'CANTINE', 'INTERNAT', 'RPI',
       'EP', 'CL_PGE', 'SECT', 'ACCES_AIRE_PRATIQUE', 'ACCES_LIBRE',
       'ACCES_SANITAIRE', 'ACCES_VESTIAIRE', 'CAPACITE_D_ACCUEIL',
       'PRES_DOUCHE', 'PRES_SANITAIRE', 'SAISONNIER', 'COUVERT', 'ECLAIRE',
       'CATEGORIE', 'MULTIPLEXE', 'ACCUEIL', 'ITINERANCE', 'MODE_GESTION',
       'SSTYPHEB', 'TYPE', 'CAPACITE', 'INDIC_CAPA', 'NBEQUIDENT',
       'INDIC_NBEQUIDENT', 'NBSALLES', 'INDIC_NBSALLES', 'NBLIEUX',
       'INDIC_NBLIEUX', 'LAMBERT_X', 'LAMBERT_Y', 'LONGITUDE', 'LATITUDE',
       'QUALITE_XY', 'EPSG', 'QUALITE_GEOLOC', 'TR_DIST_PRECISION', 'DCIRIS',
       'QUALI_IRIS', 'IRISEE', 'QP2015', 'QUALI_QP2015', 'QP', 'QUALI_QP',
       'QVA', 'QUALI_QVA', 'ZUS', 'QUALI_ZUS', 'EPCI', 'UU2020', 'BV2022',
       'AAV2020', 'DENS3', 'DENS7', 'geometry'],
 

Le dictionnaire de ces variables (colonnes) est disponible dans le fichier `BPE23_anonymisee_dictionnaire_variables.html` dans le dossier `data/raw/docs` du projet.

## La qualité de géolocalisation des équipements

Les équipements de la BPE sont géolocalisés avec différents niveaux de précision, indiqués dans la colonne `QUALITE_XY` du GeoDataFrame. Voici les différentes qualités de géolocalisation possibles :

In [4]:
print(data['QUALITE_XY'].unique())

['A' 'B' 'M' '_U']


**DESCRIPTION QUALITE_XY** 

  | Code | Libellé | Description |
  |------|---------|-------------|
  | B    | Bonne   | Précision < 100 m (adresse exacte ou voie exacte, précision < 100 m) |
  | A    | Acceptable | Précision [100 - 500[ m (voie exacte, précision entre 100 et 500 m) |
  | M    | Mauvaise | Précision ≥ 500 m (voie exacte, précision ≥ 500 m ou voie inconnue/probable, position aléatoire dans la commune) |
  | _Z   | Non-géolocalisé | Non-géolocalisé |
  | _U   | Indéterminée | Indéterminée |



Dans mon analyse, je ne vais retenir que les équipements avec une qualité de géolocalisation "Bonne" ou "Acceptable", c'est-à-dire ceux qui ont une précision de localisation inférieure à **500 mètres**. Les équipements avec une qualité "Mauvaise", "Non-géolocalisé" ou "Indéterminée" sont exclus de l'analyse, car leur localisation est trop imprécise pour être utile par rapport à granulometrie de notre analyse.

In [5]:
print("Qualité des données génerale")
print(data['QUALITE_XY'].unique())

print('-----------------------------------')

data = data[data['QUALITE_XY'].isin(['B', 'A'])]
print("Qualité des données prises")
print(data['QUALITE_XY'].unique())

Qualité des données génerale
['A' 'B' 'M' '_U']
-----------------------------------
Qualité des données prises
['A' 'B']


## Catégorisation par types d’équipements

Les équipements seront regrouper en grandes catégories (santé, éducation, culture, sport, etc.).
Cette catégorisation sera basée sur la colonne `TYPEQU` du GeoDataFrame, qui indique le type d’équipement.

La correspondance entre les codes `TYPEQU` et leur signification est disponible dans ce fichier: [https://www.insee.fr/fr/metadonnees/source/fichier/BPE23_anonymisee_dictionnaire_variables.html](https://www.insee.fr/fr/metadonnees/source/fichier/BPE23_anonymisee_dictionnaire_variables.html).

La sélection des catégories peut être adaptée selon les besoins de l’analyse.

In [6]:
print("Affichage des types d'equipements")
print(data["TYPEQU"].unique())

Affichage des types d'equipements
['A120' 'A128' 'A129' 'A203' 'A205' 'A206' 'A208' 'A301' 'A303' 'A304'
 'A401' 'A402' 'A403' 'A404' 'A405' 'A406' 'A501' 'A502' 'A503' 'A504'
 'A505' 'A506' 'A507' 'B105' 'B201' 'B202' 'B204' 'B207' 'B208' 'B209'
 'B210' 'B302' 'B303' 'B304' 'B306' 'B307' 'B309' 'B311' 'B312' 'B313'
 'B315' 'B316' 'B320' 'B321' 'B323' 'B325' 'C107' 'C108' 'C109' 'D104'
 'D109' 'D201' 'D203' 'D207' 'D208' 'D211' 'D221' 'D233' 'D235' 'D236'
 'D237' 'D238' 'D240' 'D242' 'D243' 'D244' 'D302' 'D303' 'D307' 'D401'
 'D403' 'D502' 'D505' 'D603' 'D604' 'D606' 'E101' 'F307' 'F314' 'G101'
 'G102' 'G104' 'G105' 'A101' 'A104' 'A122' 'A124' 'A132' 'A133' 'A207'
 'A302' 'B103' 'B104' 'B205' 'B206' 'B308' 'B310' 'B317' 'B318' 'B319'
 'B322' 'B324' 'C201' 'C301' 'C302' 'C402' 'C409' 'C501' 'C502' 'C601'
 'C609' 'D101' 'D102' 'D106' 'D107' 'D108' 'D111' 'D112' 'D113' 'D202'
 'D206' 'D209' 'D210' 'D212' 'D213' 'D214' 'D231' 'D239' 'D402' 'D503'
 'D504' 'D602' 'D605' 'D709' 'E108' 'F303' 

### Domaines des types d’équipements

Chaque code correspond à un type d’équipement, réparti en **7 grands domaines** :

| Code | Domaine d’appartenance         |
| ---- | ------------------------------ |
| A    | Services pour les particuliers |
| B    | Commerces                      |
| C    | Enseignement                   |
| D    | Santé et action sociale        |
| E    | Transports et déplacements     |
| F    | Sports, loisirs et culture     |
| G    | Tourisme                       |


### Exemple : catégorisation du domaine **Santé**

Pour illustrer la démarche, prenons l’exemple du domaine **Santé**.
Dans cette catégorie, on peut retenir différents équipements en fonction des objectifs d’analyse.

Par exemple : médecin généraliste, masseur-kinésithérapeute, laboratoire d’analyses médicales, pharmacie ou encore centre de santé.

Dans le dictionnaire de variables de la BPE, cela se revient à prendre :

In [7]:
health = {
    'D108', 'D113', 'D201', 'D233', 'D302', 'D307'
}

Ici, chaque code est associé à un type d’équipement :

* `D108` : Centre de santé
* `D113` : Maison de santé pluridisciplinaire
* `D201` : Médecin généraliste
* `D233` : Masseur-kinésithérapeute
* `D302` : Laboratoire d’analyses médicales
* `D307` : Pharmacie

## Pondération des équipements

En plus de la catégorisation, les équipements vont être pondérés selon deux dimensions :

* leur importance (poids intrinsèque),
* leur environnement (poids extrinsèque).

Pour cela, nous créerons deux colonnes dans le GeoDataFrame :

* `wi` pour le poids intrinsèque,
* `wecat` pour le poids extrinsèque.

### Crétation des colonnes et initialisation

In [8]:
data["cat"] = "tbd"  # une chaîne de caractères (categorie sociale)
data["wi"] = 0.0    # un float (poids intrinsèque)
data["wecat"] = 0.0  # un float (poids extrinsèque)

# Vérification si colonnes ont été ajoutées 
data.head()

,AN,NOMRS,CNOMRS,NUMVOIE,INDREP,TYPVOIE,LIBVOIE,CADR,LIBCOM,CODPOS,...,EPCI,UU2020,BV2022,AAV2020,DENS3,DENS7,geometry,cat,wi,wecat
0,2023,DIR REG FIP PACA ET BOUCHES DU RHONE,TRESORERIE ALLAUCH,,,TRA,LA CHAP DES FILLES,,ALLAUCH,13190,...,200054807,00759,13055,003,1,1,POINT (901350.000 6251550.000),tbd,0.0,0.0
1,2023,FRANCE SERVICES D’ALLAUCH - PLAN DE CUQUES,,17,,BD,CLEMENT ET JULES BARTHELEMY,,ALLAUCH,13190,...,200054807,00759,13055,003,1,1,POINT (899921.860 6251277.510),tbd,0.0,0.0
2,2023,ALLAUCH,,1,,PL,DOCTEUR JOSEPH CHEVILLON,,ALLAUCH,13190,...,200054807,00759,13055,003,1,1,POINT (901355.700 6251704.700),tbd,0.0,0.0
3,2023,CAISSE D'EPARGNE CEPAC,AGENCE ALLAUCH CANTON VERT,,,AV,DU CANTON VERT,CC ESPACE MELVILLE,ALLAUCH,13190,...,200054807,00759,13055,003,1,1,POINT (900461.000 6252323.800),tbd,0.0,0.0
4,2023,BANQUE POPULAIRE MEDITERRANEE,,,,AV,LEI RIMA,LE LOGIS NEUF,ALLAUCH,13190,...,200054807,00759,13055,003,1,1,POINT (901698.290 6254031.270),tbd,0.0,0.0


Ensuite je réduis le GeoDataFrame aux colonnes essentielles pour l'analyse, à savoir :

In [9]:
data = data[['TYPEQU', 'geometry', 'cat', 'wi', 'wecat']] # On réduit à 5 colonnes + l'ID

In [10]:
# Affichage des informations sur les colonnes retenues
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 54113 entries, 0 to 58686
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   TYPEQU    54113 non-null  object  
 1   geometry  54113 non-null  geometry
 2   cat       54113 non-null  object  
 3   wi        54113 non-null  float64 
 4   wecat     54113 non-null  float64 
dtypes: float64(2), geometry(1), object(2)
memory usage: 2.5+ MB


### Remplissage des colonnes `cat`, `wi` et `wecat`

* La colonne `cat` (catégorie sociale) est remplie à partir du type d’équipement (`TYPEQU`).
  Par exemple, les équipements de santé sont marqués comme *"santé"*, ceux d’éducation comme *"éducation"*, etc.

* La colonne `wi` (poids intrinsèque) représente l’importance relative de chaque type d’équipement à l’intérieur de sa catégorie.

* La colonne `wecat` (poids extrinsèque) correspond au contexte : elle prend en compte la diversité des services disponibles autour de l’équipement.

Dans cette partie, seule la colonne `cat` sera renseigner.
Les colonnes `wi` et `wecat` sont initialisées à zéro et feront l’objet d’analyses ultérieurs. Pour illustrer, prenons l’exemple du domaine Santé.

In [11]:
# Normaliser les valeurs dans la base
data['TYPEQU'] = data['TYPEQU'].str.strip().str.upper()

data.loc[data['TYPEQU'].isin(health), 'cat'] = 'health'

# Supprimer les lignes qui ne sont pas dans les listes
data = data[data['cat'] != 'tbd']

In [12]:
data["TYPEQU"].unique()

array(['D201', 'D233', 'D302', 'D307', 'D108', 'D113'], dtype=object)

In [13]:
# Réinitialisation de l'index 
data = data.reset_index(drop=True)

# Affichage de l'entête du tableau
data.head()

,TYPEQU,geometry,cat,wi,wecat
0,D201,POINT (901648.360 6253959.190),health,0.0,0.0
1,D201,POINT (901650.000 6253850.000),health,0.0,0.0
2,D201,POINT (901387.000 6251711.700),health,0.0,0.0
3,D201,POINT (901236.730 6251635.020),health,0.0,0.0
4,D201,POINT (901244.210 6251911.890),health,0.0,0.0


In [14]:
# Affichage des informations 
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4670 entries, 0 to 4669
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   TYPEQU    4670 non-null   object  
 1   geometry  4670 non-null   geometry
 2   cat       4670 non-null   object  
 3   wi        4670 non-null   float64 
 4   wecat     4670 non-null   float64 
dtypes: float64(2), geometry(1), object(2)
memory usage: 182.5+ KB


## Sauvegarde des données

Le GeoDataFrame prétraité est sauvegardé dans un nouveau fichier GeoPackage.
Ce fichier contient les colonnes essentielles qui serviront de base pour les analyses des sections suivantes.


In [15]:
# Sauvegarde de la nouvelle base
data.to_file('../../proxy/data/processed/b3_2501171030_health_mrs.gpkg', driver='GPKG')

## Conclusion

Dans cette partie, j’ai montré les premières étapes de préparation des données de la **BPE 2023** : importation, filtrage selon la qualité de la géolocalisation et illustration de la catégorisation possible des équipements.
J’ai également introduit la logique de pondération (intrinsèque et extrinsèque), qui sera développée dans les analyses suivantes.

Le GeoDataFrame sauvegardé ici constitue donc une base de travail, encore partielle, mais déjà utile pour avancer dans la suite du projet.